In [1]:
from google.colab import drive
drive.mount('/content/gdrive')
import os

Mounted at /content/gdrive


In [2]:
import pandas as pd
import json
import glob
import ast
from tqdm.auto import tqdm
import numpy as np

In [3]:
files = glob.glob("/content/gdrive/MyDrive/Colab/COMS6998EMODS/pannel_data/Hate_Rate_by_State/*")
cnt = 0
for file in files:
  df = pd.read_csv(file)
  cnt += df['hate_cnt'].sum()
cnt 

24295

In [4]:
df = pd.read_csv('/content/gdrive/MyDrive/Colab/COMS6998EMODS/pannel_data/Day_panel/2020-08-03.csv')
df

,state,positiveIncreaseRate,deathIncreaseRate,negativeIncreaseRate,hospitalizedIncreaseRate,totalTestResultsIncreaseRate,trumpNegRate,trumpPosRate,hateSpeechRate
0,AZ,-29.692833,-22.222222,10.248693,16.000000,-42.957926,2.463108,-0.963802,100.000000
1,CA,-36.459256,-75.757576,0.000000,0.000000,-0.446488,2.463108,-0.963802,-10.000000
2,FL,-31.774376,17.741935,-21.483309,22.222222,-30.160631,2.463108,-0.963802,46.666667
3,GA,-28.657188,-86.666667,0.000000,-13.043478,-24.312580,2.463108,-0.963802,36.363636
4,IL,-11.520109,-35.714286,0.000000,0.000000,-26.884067,2.463108,-0.963802,-56.250000
5,NV,-12.113174,inf,-74.098700,0.000000,-33.278956,2.463108,-0.963802,14.285714
6,NY,2.636535,-66.666667,0.000000,0.000000,-12.079171,2.463108,-0.963802,48.000000
7,OH,-1.271186,-28.571429,0.000000,113.953488,-19.892433,2.463108,-0.963802,125.000000
8,PA,-13.608563,-100.000000,-1.362891,0.000000,-4.667189,2.463108,-0.963802,14.285714
9,TX,-14.824928,inf,0.000000,0.000000,-22.561248,2.463108,-0.963802,-3.125000


In [ ]:
DATES = open("/content/gdrive/MyDrive/Colab/COMS6998EMODS/dates.txt").readlines()
DATES = [date.split("\n")[0] for date in DATES]

DIR = '/content/gdrive/MyDrive/Colab/COMS6998EMODS/Hate_Tweet_0.01/*'
files = glob.glob(DIR)

cnt = 0
for pfile in tqdm(files):

  df = pd.read_csv(pfile, engine='python')
  Hate_cnt = []
  if df.shape[0] > 1000:
    cnt += 1
    print("\""+str(pfile.split("/")[-1].split(".")[0]) + "\"")

    day_cnt = 0
    new_date = []

    for date in DATES:
      if date in ['2020-08-02', '2020-08-04', '2020-08-05', '2020-08-07']:
        continue
      new_date.append(date)
      daily = df[df['date'] == date]
      row_cnt = daily.shape[0]
      if row_cnt > 0:
        Hate_cnt.append(row_cnt)
      else:
        Hate_cnt.append(1)
    state_df = pd.DataFrame(list(zip(new_date, Hate_cnt)), columns =['date', 'hate_cnt'])
    state_df['rate'] = state_df['hate_cnt'].pct_change() * 100
    state_df.to_csv("/content/gdrive/MyDrive/Colab/COMS6998EMODS/pannel_data/Hate_Rate_by_State/" + pfile.split("/")[-1], index=False)

state_df

  0%|          | 0/59 [00:00<?, ?it/s]

"AZ"
"CA"
"FL"
"GA"
"IL"
"NV"
"NY"
"OH"
"PA"
"TX"
"WA"


,date,hate_cnt,rate
0,2020-08-01,15,NaN
1,2020-08-03,20,33.333333
2,2020-08-06,5,-75.000000
3,2020-08-08,11,120.000000
4,2020-08-09,10,-9.090909
...,...,...,...
144,2020-12-27,2,-50.000000
145,2020-12-28,6,200.000000
146,2020-12-29,6,0.000000
147,2020-12-30,7,16.666667


In [ ]:
state_code = pd.read_csv('/content/gdrive/MyDrive/Colab/COMS6998EMODS/state_code.csv')
code_dict = {}
for index, row in state_code.iterrows():
  code_dict[row['code']] = row['name'].lower()
code_dict

In [ ]:
Covid_DIR = '/content/gdrive/MyDrive/Colab/COMS6998EMODS/CovidData/States_Data_Rate/'

DEST = '/content/gdrive/MyDrive/Colab/COMS6998EMODS/pannel_data/Day_panel/'
SRC = "/content/gdrive/MyDrive/Colab/COMS6998EMODS/pannel_data/Hate_Rate_by_State/"

Selceted_States = ["AZ", "CA","FL","GA","IL","NV","NY","OH", "PA", "TX", "WA"]
for date in tqdm(new_date):

  Hate_rate = []
  case = []
  death = []
  negSamp = []
  hosp = []
  test = []
  trumpNeg = []
  trumpPos = []

  for state in Selceted_States:
    
    #loading hate speech rate
    path = SRC + state + ".csv"

    df = pd.read_csv(path, engine='python')
    df = df.set_index('date')

    Hate_rate.append(df.loc[date]['rate'])


    # loading covid data: 5 factors
    covid_path = Covid_DIR + state + ".csv"
    
    covid_df = pd.read_csv(covid_path, engine='python')
    covid_df = covid_df.set_index('date')

    case.append(covid_df.loc[date]['positiveIncreaseRate'])
    death.append(covid_df.loc[date]['deathIncreaseRate'])
    negSamp.append(covid_df.loc[date]['negativeIncreaseRate'])
    hosp.append(covid_df.loc[date]['hospitalizedIncreaseRate'])
    test.append(covid_df.loc[date]['totalTestResultsIncreaseRate'])

    # loading trump data
    trump_df = pd.read_csv("/content/gdrive/MyDrive/Colab/COMS6998EMODS/trump_score.csv", engine='python')
    trump_df = trump_df.set_index('date')

    trumpNeg.append(trump_df.loc[date]['NegRate'])
    trumpPos.append(trump_df.loc[date]['PosRate'])

  # merge all data into data panel
  total_df = pd.DataFrame(list(zip(Selceted_States, case, death, negSamp, hosp, test, trumpNeg, trumpPos, Hate_rate)), \
                          columns =['state', 'positiveIncreaseRate', 'deathIncreaseRate', 'negativeIncreaseRate', 'hospitalizedIncreaseRate',\
                                    'totalTestResultsIncreaseRate', 'trumpNegRate', 'trumpPosRate', 'hateSpeechRate'])
  total_df.to_csv(DEST + date + ".csv", index=False)
total_df



  0%|          | 0/149 [00:00<?, ?it/s]

,state,positiveIncreaseRate,deathIncreaseRate,negativeIncreaseRate,hospitalizedIncreaseRate,totalTestResultsIncreaseRate,trumpNegRate,trumpPosRate,hateSpeechRate
0,AZ,46.535029,87.179487,13.873556,-33.286319,32.843510,-99.645626,24.498692,0.000000
1,CA,-11.914233,-0.925926,0.000000,0.000000,-6.515959,-99.645626,24.498692,-46.666667
2,FL,23.383194,-4.316547,57.335997,-2.659574,-15.195636,-99.645626,24.498692,-66.666667
3,GA,29.338341,31.343284,0.000000,-18.400000,89.685679,-99.645626,24.498692,-50.000000
4,IL,8.611337,-22.325581,0.000000,0.000000,33.327075,-99.645626,24.498692,-14.285714
5,NV,-13.481781,25.531915,36.589862,0.000000,19.390385,-99.645626,24.498692,-62.500000
6,NY,25.182536,-9.395973,0.000000,0.000000,39.779540,-99.645626,24.498692,-35.294118
7,OH,17.779408,-19.548872,0.000000,-9.289617,131.824289,-99.645626,24.498692,0.000000
8,PA,0.089047,-4.075235,6.258583,0.000000,9.506516,-99.645626,24.498692,22.222222
9,TX,-12.781219,7.055215,0.000000,0.000000,16.493428,-99.645626,24.498692,-5.882353
